In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
google_api_key = os.getenv('GOOGLE_API_KEY')
if not google_api_key: 
    raise ValueError("GOOGLE_API_KEY is not set in the environment variables.")

In [2]:
from langchain.chat_models import init_chat_model
llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [3]:
from langchain.agents import create_agent

def get_weather(city: str) -> str:  
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_agent(
    model=llm,  
    tools=[get_weather],  
    system_prompt="You are a helpful assistant"  
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in San Francisco?"}]}
)

{'messages': [HumanMessage(content='what is the weather in San Francisco?', additional_kwargs={}, response_metadata={}, id='6587adcb-d225-4933-a1be-c18acf4870b7'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"city": "San Francisco"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--f4f9a6de-3528-4ccf-bf04-8a0a68a88077-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': '8e93a58f-aeed-4a65-b8a1-9f113422ad7e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 27, 'output_tokens': 6, 'total_tokens': 33, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content="It's always sunny in San Francisco!", name='get_weather', id='7e26ad5d-d1b0-4597-a86a-126927129438', tool_call_id='8e93a58f-aeed-4a65-b8a1-9f113422ad7e'),
  AIMessage(content=

In [4]:
agent = create_agent(
    model=llm,  
    tools=[get_weather],  
    system_prompt="Never answer questions about the weather."  
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in San Francisco?"}]}
)

{'messages': [HumanMessage(content='what is the weather in San Francisco?', additional_kwargs={}, response_metadata={}, id='2e8161e2-1b77-43e0-9f31-a9fce3a5f205'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"city": "San Francisco"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--6c3ba6a4-63c6-42b4-9a61-969408671873-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 'e9244da0-5dbc-42c8-abd2-76c789cb92e7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 29, 'output_tokens': 6, 'total_tokens': 35, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content="It's always sunny in San Francisco!", name='get_weather', id='bea3546a-db48-4f33-9835-c54d2bf19d52', tool_call_id='e9244da0-5dbc-42c8-abd2-76c789cb92e7'),
  AIMessage(content=

In [5]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

agent = create_agent(
    model=llm,
    tools=[get_weather],
    checkpointer=checkpointer  
)

# Run the agent
config = {"configurable": {"thread_id": "1"}}
sf_response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in san francisco?"}]},
    config  
)
ny_response = agent.invoke(
    {"messages": [{"role": "user", "content": "what about new york?"}]},
    config
)

In [6]:
sf_response, ny_response

({'messages': [HumanMessage(content='what is the weather in san francisco?', additional_kwargs={}, response_metadata={}, id='41c5eb43-3574-4403-8a9e-a5ff2d2ab075'),
   AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"city": "san francisco"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--1d707f2c-3cb1-4b28-b829-eea26fba2b69-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'san francisco'}, 'id': '546a4514-afbf-4079-9184-59085cd27f2a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 22, 'output_tokens': 6, 'total_tokens': 28, 'input_token_details': {'cache_read': 0}}),
   ToolMessage(content="It's always sunny in san francisco!", name='get_weather', id='189d5e87-cb61-4a0f-9ad2-d3045207ded9', tool_call_id='546a4514-afbf-4079-9184-59085cd27f2a'),
   AIMessage(cont

In [7]:
from pydantic import BaseModel

class WeatherResponse(BaseModel):
    city: str
    conditions: str

agent = create_agent(
    model=llm,
    tools=[get_weather],
    response_format=WeatherResponse  
)
response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in san francisco?"}]}
)

response["structured_response"]

WeatherResponse(city='san francisco', conditions='sunny')